<a href="https://colab.research.google.com/github/rgprado/Mineracao_de_Texto_na_Web_Coleta_de_Dados/blob/main/Atividade_1_Coleta_de_dados_na_Web%2C_Pr%C3%A9_processamento_e_Extra%C3%A7%C3%A3o_de_Caracter%C3%ADsticas_do_Texto_Rodrigo_Prado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Coleta de dados na Web, Pré processamento e Extração de Características do Texto##
Professor: Leonardo Valeriano Neri

Aluno: Rodrigo Prado

Atividade 1 


In [1]:
!pip install beautifulsoup4
!conda activate NLP
!pip install nltk
!pip install --upgrade gensim
!pip install sklearn
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação das bibliotecas

In [2]:
import pandas as pd
import numpy as np
import json
from typing import List
import os.path
from os import path

import os
import glob
import re
from unidecode import unidecode

from bs4 import BeautifulSoup
import requests
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Função para criar os diretórios para salvar os arquivos extraidos

In [3]:
def create_json_directory(path: str):
  output_path = "output"

  if not os.path.exists(output_path):
    os.makedirs(output_path)

  if not os.path.exists(output_path + '/' + path):
    os.makedirs(output_path + '/' + path)
  else:
    print("Path already exists")

## Função para Coleta de dados na Web

In [4]:
def read_page_get_links(keywords: str) -> list:
    response = requests.get("https://repositorio.ufpe.br/simple-search?location=123456789%2F50&query={}".format(keywords))
    soup = BeautifulSoup(response.text.strip(), 'html.parser')
    tags = soup.find_all(name='table')
    tag_table = tags[0]
    tags_a = tag_table.find_all(name='a')
    
    urls = []
    
    for i in range(0, len(tags_a), 2): 
        link = tags_a[i]["href"]
        urls.append(link)
    
    return urls

## Função para Leitura do titulo e abstract

In [5]:
def read_table_get_title_and_abstract(info_table: List[str]) -> dict:
    all_titles = info_table.find_all(name='td', attrs={'class': 'dc_title'})
    title = all_titles[-1].contents[0]

    all_abst = info_table.find_all(name='td', attrs={'class': 'dc_description_abstract'})
    abst = all_abst[-1].contents[0]

    dict_thesis_info = {}
    dict_thesis_info = {
        'title': title,
        'abstract': abst,
    }

    return dict_thesis_info

## Função para Criar uma tabela com todos os artigos

In [6]:
def create_tables(urls):
  info_table = []

  for url in urls:
      response = requests.get(f'https://repositorio.ufpe.br/{url}', allow_redirects=True)
      soup = BeautifulSoup(response.text.strip(), 'html.parser')
      tags_table = soup.find_all(name='table', attrs={'class': 'itemDisplayTable'})
      info_table.append(tags_table[0])

  return info_table

## Função para Salvar todos os artigos por tema no diretório

In [7]:
def create_output_folder(topic: str, info_table):
  counter = 1

  create_json_directory(topic)

  for table in info_table:
      output = read_table_get_title_and_abstract(table)
      
      # Criar a estrutura de pastas antes de executar a celula
      with open(f"./output/{topic}/thesis{counter}.json", 'w') as json_file:
          json.dump(output, json_file, ensure_ascii=False) #TODO: Automatizar a criação das pastas
          
      counter = counter + 1

## Escolha dos temas a serem pesquisados

In [8]:
topic_list = ["machine+learning", "economia", "estatistica", "historia", "neuro+ciencia"]
all_urls = []

for topic in topic_list:
  urls = read_page_get_links(topic)
  info_table = create_tables(urls)
  create_output_folder(topic, info_table)

Path already exists
Path already exists
Path already exists
Path already exists
Path already exists


In [9]:
path = os.getcwd()
files = glob.glob(os.path.join(path, "output/*/*.json"))

In [10]:
jsons = [pd.read_json(file, lines=True) for file in files]
thesis = pd.concat(jsons, ignore_index=True)

thesis["subject"] = [os.path.split(os.path.dirname(file))[-1] for file in files]

In [11]:
thesis

,title,abstract,subject
0,Metodologia de design aplicada à concepção de ...,Da década de 1990 até o momento a tecnologia d...,historia
1,Saberes docentes do professor de História mobi...,Esta pesquisa trata sobre os saberes docentes ...,historia
2,O ensino da história: Outros recursos além do ...,Este trabalho engloba um levantamento e anális...,historia
3,Vozes de clio: um estudo sobre o ensino de His...,Este trabalho analisa as condições atuais do e...,historia
4,A história local no currículo da educação básica,O Ensino de História do Recife foi instituído ...,historia
5,O direito ao passado : uma discussão necessári...,O objetivo deste trabalho é demonstrar a neces...,historia
6,Trajetórias invisibilizadas : Matilde Landeta ...,"A partir da década de 1980, com a apropriação ...",historia
7,Biografia na sala de aula : a construção de sa...,O presente trabalho estuda possibilidades de u...,historia
8,O tempo histórico na representação dos Profes...,Objetiva analisar a representação de tempo par...,historia
9,Contações de história na região do Cariri Cear...,Considera que a memória oral foi por muito tem...,historia


## Definição das StopWords

In [12]:
languages = ['english', 'portuguese']
list_stop = stopwords.words(languages)
print('Both languages Stop Words', list_stop)

Both languages Stop Words ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own

In [13]:
texto = ''
for doc in thesis[:1].iterrows():
  texto = doc[1]['title'] + ' ' + doc[1]['abstract']

print(texto)
print(thesis[:1]['abstract'])

Metodologia de design aplicada à concepção de histórias em quadrinhos digitais Da década de 1990 até o momento a tecnologia digital vem mudando de forma
radical quase todos os setores da sociedade. Vivemos num período de adaptação ao
uso desta tecnologia. Não distantes dessa revolução, a linguagem conhecida
popularmente como histórias em quadrinhos também migra para este novo
ambiente.
Existem várias manifestações de histórias em quadrinhos digitais, inclusive
produzidas por grandes editoras tradicionais, no entanto a maioria destes artefatos
ainda estão presos a características dadas pela imprensa, principalmente o formato
tradicional de página. Estudiosos no assunto acreditam que as histórias em
quadrinhos no ambiente digital devem apresentar soluções distintas de sua forma
na imprensa, criando experiências interativas, imersivas, novas formas de leitura,
mas sem perder as características de sua linguagem.
Neste trabalho partimos do pressuposto que as histórias em quadrinhos digitais

In [14]:
documents_tokens = []
vocabulary = set()
for article in thesis.iterrows():
  doc = article[1]['title'] + ' ' + article[1]['abstract']
  doc_tokens = [
    token.lower() # pre processamento: todas as letras ficam minúsculas
      for token in word_tokenize(doc, language='portuguese') # tokenizador para o idioma inglês a nível de palavras
        if not token.lower() in list_stop and token.isalnum() # excluir tokens na lista "stop" e símbolos
  ]

  # doc_tokens = []
  # for token in word_tokenize(doc, language='english'):
  #   token = token.lower()
  #   if not token in list_stop:
  #     if not token.isalnum():
  #       doc_tokens.append(token)

  vocabulary.update(set(doc_tokens))
  documents_tokens.append(doc_tokens)

In [15]:
len(vocabulary)

3543

In [16]:
for v in list(vocabulary):
  if len(v) == 1:
    print(v)

7
8
5
1
9
3
k
p
g
4
2
n
b
x
c
á


In [17]:
documents_tokens[0]

['metodologia',
 'design',
 'aplicada',
 'concepção',
 'histórias',
 'quadrinhos',
 'digitais',
 'década',
 '1990',
 'momento',
 'tecnologia',
 'digital',
 'vem',
 'mudando',
 'forma',
 'radical',
 'quase',
 'todos',
 'setores',
 'sociedade',
 'vivemos',
 'período',
 'adaptação',
 'uso',
 'desta',
 'tecnologia',
 'distantes',
 'dessa',
 'revolução',
 'linguagem',
 'conhecida',
 'popularmente',
 'histórias',
 'quadrinhos',
 'migra',
 'novo',
 'ambiente',
 'existem',
 'várias',
 'manifestações',
 'histórias',
 'quadrinhos',
 'digitais',
 'inclusive',
 'produzidas',
 'grandes',
 'editoras',
 'tradicionais',
 'entanto',
 'maioria',
 'destes',
 'artefatos',
 'ainda',
 'presos',
 'características',
 'dadas',
 'imprensa',
 'principalmente',
 'formato',
 'tradicional',
 'página',
 'estudiosos',
 'assunto',
 'acreditam',
 'histórias',
 'quadrinhos',
 'ambiente',
 'digital',
 'devem',
 'apresentar',
 'soluções',
 'distintas',
 'forma',
 'imprensa',
 'criando',
 'experiências',
 'interativas',
 '

In [18]:
print(thesis[:1])
pre_processed_documents = [' '.join(doc) for doc in documents_tokens]
print(pre_processed_documents[0])

                                               title  \
0  Metodologia de design aplicada à concepção de ...   

                                            abstract   subject  
0  Da década de 1990 até o momento a tecnologia d...  historia  
metodologia design aplicada concepção histórias quadrinhos digitais década 1990 momento tecnologia digital vem mudando forma radical quase todos setores sociedade vivemos período adaptação uso desta tecnologia distantes dessa revolução linguagem conhecida popularmente histórias quadrinhos migra novo ambiente existem várias manifestações histórias quadrinhos digitais inclusive produzidas grandes editoras tradicionais entanto maioria destes artefatos ainda presos características dadas imprensa principalmente formato tradicional página estudiosos assunto acreditam histórias quadrinhos ambiente digital devem apresentar soluções distintas forma imprensa criando experiências interativas imersivas novas formas leitura perder características linguagem nes

In [19]:
custom_list_stop = []
for v in list(vocabulary):
  if len(v) == 1 or len(v) == 2:
    custom_list_stop.append(v)

custom_list_stop = custom_list_stop + ['xterm', 'x11r5', 'x11', 'xlib', 'vax', 'uwo', 'pl8', 'pl9']
print(custom_list_stop)

del documents_tokens
del vocabulary
documents_tokens = []
vocabulary = set()
for doc in pre_processed_documents:
    tokens = [token for token in word_tokenize(doc, language='portuguese') if not token in custom_list_stop]
    documents_tokens.append(tokens)
    vocabulary.update(set(tokens))

print('Tamanho do novo vocabulário: ', len(vocabulary))

del pre_processed_documents
pre_processed_documents = [' '.join(doc) for doc in documents_tokens]

['52', '2º', 'nº', 't2', 'ph', '7', 'rs', '60', '8', 'cd', '42', '12', '56', '5', '1', '72', '9', '3', 'gc', 'qm', 'es', '11', '85', 'k', 'p', 'fe', 'ar', '20', 'xx', 'g', '4', '15', 'pn', '70', '2', 'ex', '30', 'ir', '48', '1º', '19', '17', 'ps', 'aí', '14', '18', '98', 'wi', 'n', 'b', 'lm', '65', 'mg', '32', '10', 'x', 'je', 'ii', 'dp', 'f1', 'ee', '95', '78', '81', 'ia', 'si', 'la', '16', '50', 'sm', 'mm', 'ml', '90', 'dl', '80', 'dá', 'pe', 'c', 'á', 'xterm', 'x11r5', 'x11', 'xlib', 'vax', 'uwo', 'pl8', 'pl9']
Tamanho do novo vocabulário:  3464


In [20]:
for v in list(vocabulary):
  if len(v) == 3:
    print(v)

snh
pca
vem
dar
vêm
shi
mão
vgg
deu
dão
two
tcm
iii
bad
snc
edc
end
pib
hiv
luz
mês
one
imc
rna
hal
dsl
têm
web
new
adu
sim
rio
für
lee
avd
boa
180
aim
ips
ies
fim
955
diz
vez
set
fez
ano
bem
dor
tal
klm
faz
dez
vão
cpe
ktk
way
631
vai
águ
gal
may
rel
elm
uso
sul
142
mlp
car
adl
via
kdd
sob
lei
dia
svm
bpn
rbf
bom
oil
ide
knn
fwd
sgd
cnn
200
ter


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(stop_words=list_stop, min_df=0.005, max_df=0.5, max_features=1000)
count_vect.fit(pre_processed_documents)

CountVectorizer(max_df=0.5, max_features=1000, min_df=0.005,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [22]:
print('Tamanho do vocabulário: ', len(count_vect.vocabulary_))
counter = 0
for _, count in count_vect.vocabulary_.items():
    counter += count
    
print('Quantidade total de palavras: ', counter)

Tamanho do vocabulário:  1000
Quantidade total de palavras:  499500


In [23]:
sorted(count_vect.vocabulary_.items(), reverse=True)

[('única', 999),
 ('última', 998),
 ('ótica', 997),
 ('órgão', 996),
 ('índices', 995),
 ('índice', 994),
 ('ênfase', 993),
 ('ética', 992),
 ('âmbito', 991),
 ('árvore', 990),
 ('áreas', 989),
 ('área', 988),
 ('água', 987),
 ('zoologia', 986),
 ('zika', 985),
 ('yahr', 984),
 ('xukuru', 983),
 ('works', 982),
 ('work', 981),
 ('web', 980),
 ('way', 979),
 ('vários', 978),
 ('visão', 977),
 ('vista', 976),
 ('vida', 975),
 ('vez', 974),
 ('vetores', 973),
 ('versões', 972),
 ('verificar', 971),
 ('verbal', 970),
 ('vem', 969),
 ('velocidade', 968),
 ('vector', 967),
 ('variáveis', 966),
 ('valores', 965),
 ('utilização', 964),
 ('utilizando', 963),
 ('utilizam', 962),
 ('utilizados', 961),
 ('utilizado', 960),
 ('utilizadas', 959),
 ('utilizada', 958),
 ('usuário', 957),
 ('usually', 956),
 ('uso', 955),
 ('using', 954),
 ('useful', 953),
 ('used', 952),
 ('usados', 951),
 ('usado', 950),
 ('universidade', 949),
 ('unified', 948),
 ('understanding', 947),
 ('ufpe', 946),
 ('típica', 9

In [24]:
count_vect.vocabulary_.items()

dict_items([('metodologia', 565), ('design', 237), ('aplicada', 42), ('concepção', 171), ('histórias', 464), ('quadrinhos', 775), ('digitais', 263), ('década', 289), ('1990', 3), ('momento', 584), ('tecnologia', 898), ('digital', 264), ('vem', 969), ('forma', 428), ('quase', 782), ('todos', 922), ('setores', 857), ('sociedade', 875), ('período', 690), ('uso', 955), ('desta', 243), ('dessa', 239), ('novo', 634), ('ambiente', 31), ('entanto', 315), ('maioria', 534), ('destes', 246), ('ainda', 19), ('características', 127), ('principalmente', 728), ('formato', 430), ('tradicional', 928), ('estudiosos', 358), ('assunto', 85), ('devem', 252), ('apresentar', 65), ('experiências', 384), ('novas', 632), ('formas', 429), ('leitura', 518), ('neste', 621), ('trabalho', 926), ('base', 106), ('metodologias', 566), ('técnicas', 942), ('novos', 635), ('profissionais', 749), ('envolvidos', 323), ('processos', 741), ('desenvolvimento', 235), ('sendo', 848), ('assim', 79), ('estudo', 359), ('fez', 415),

In [25]:
X = count_vect.transform(pre_processed_documents)

In [26]:
X.shape

(50, 1000)

In [27]:
print(X[0,:])

  (0, 3)	1
  (0, 19)	1
  (0, 31)	4
  (0, 42)	1
  (0, 48)	1
  (0, 65)	1
  (0, 79)	1
  (0, 85)	1
  (0, 106)	1
  (0, 127)	2
  (0, 171)	4
  (0, 212)	1
  (0, 225)	1
  (0, 234)	1
  (0, 235)	1
  (0, 237)	4
  (0, 239)	1
  (0, 243)	1
  (0, 246)	1
  (0, 252)	1
  (0, 263)	3
  (0, 264)	4
  (0, 289)	1
  (0, 315)	1
  (0, 323)	1
  :	:
  (0, 621)	1
  (0, 632)	1
  (0, 634)	1
  (0, 635)	1
  (0, 688)	1
  (0, 690)	1
  (0, 728)	1
  (0, 740)	2
  (0, 741)	1
  (0, 743)	1
  (0, 749)	1
  (0, 775)	7
  (0, 782)	1
  (0, 848)	2
  (0, 857)	1
  (0, 871)	1
  (0, 875)	1
  (0, 898)	2
  (0, 922)	1
  (0, 926)	2
  (0, 928)	1
  (0, 942)	1
  (0, 955)	1
  (0, 963)	1
  (0, 969)	1


In [28]:
tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X)

TfidfTransformer()

In [29]:
X_tfidf = tfidf_transformer.transform(X)
print(X_tfidf[0,:])

  (0, 969)	0.062459252264568436
  (0, 963)	0.062459252264568436
  (0, 955)	0.05140915255565674
  (0, 942)	0.056133739989525464
  (0, 928)	0.062459252264568436
  (0, 926)	0.06594009166970101
  (0, 922)	0.05903169682318615
  (0, 898)	0.13330848218143113
  (0, 875)	0.056133739989525464
  (0, 871)	0.056133739989525464
  (0, 857)	0.062459252264568436
  (0, 848)	0.08620582899071526
  (0, 782)	0.07206252231735398
  (0, 775)	0.5577954660941838
  (0, 749)	0.07206252231735398
  (0, 743)	0.07206252231735398
  (0, 741)	0.05903169682318615
  (0, 740)	0.07890575416164528
  (0, 728)	0.04763664234758206
  (0, 690)	0.056133739989525464
  (0, 688)	0.053623415596547735
  (0, 635)	0.062459252264568436
  (0, 634)	0.07206252231735398
  (0, 632)	0.05903169682318615
  (0, 621)	0.056133739989525464
  :	:
  (0, 323)	0.05903169682318615
  (0, 315)	0.05903169682318615
  (0, 289)	0.06665424109071556
  (0, 264)	0.26661696436286225
  (0, 263)	0.18737775679370533
  (0, 252)	0.06665424109071556
  (0, 246)	0.0561337399

In [30]:
X_tfidf.shape

(50, 1000)

In [31]:
# Pre processamento
final_list_stop_words = list_stop + custom_list_stop
query = 'Economia digital na era do machine learning deep learning, inteligência artificial, um panorama histórico no brasil e no mundo'

query_tokens = [token for token in word_tokenize(query, language='portuguese') if not token in final_list_stop_words]
print(query_tokens)
pre_processed_query = ' '.join(query_tokens)
print(pre_processed_query)

# Extração de características de contagem
X_query = count_vect.transform([pre_processed_query])
print('1a etapa de extração de características')
print(X_query)

# Extração de características de frequência TF-IDF
X_tfidf_query = tfidf_transformer.transform(X_query)
print('2a etapa de extração de características')
print(X_tfidf_query)

['Economia', 'digital', 'machine', 'learning', 'deep', 'learning', ',', 'inteligência', 'artificial', ',', 'panorama', 'histórico', 'brasil', 'mundo']
Economia digital machine learning deep learning , inteligência artificial , panorama histórico brasil mundo
1a etapa de extração de características
  (0, 76)	1
  (0, 118)	1
  (0, 218)	1
  (0, 264)	1
  (0, 291)	1
  (0, 466)	1
  (0, 499)	1
  (0, 517)	2
  (0, 529)	1
2a etapa de extração de características
  (0, 529)	0.27437978762070714
  (0, 517)	0.5260997347170011
  (0, 499)	0.3195909134218554
  (0, 466)	0.3410557927173024
  (0, 291)	0.2353768249413062
  (0, 264)	0.3410557927173024
  (0, 218)	0.3020528300379015
  (0, 118)	0.27437978762070714
  (0, 76)	0.3020528300379015


In [32]:
sorted_X_indices = np.argsort(X.toarray(), axis=1)[:, ::-1]
sorted_Xtfidf_indices = np.argsort(X_tfidf.toarray(), axis=1)[:,::-1]

print('Termos mais comuns nos 10 primeiros documentos:')
for i in range(10):
  print(count_vect.get_feature_names_out()[sorted_X_indices[i, :3]])

print('\nTermos mais relevantes nos 10 primeiros documentos:')
for i in range(10):
  print(count_vect.get_feature_names_out()[sorted_Xtfidf_indices[i, :3]])

print('\nTermos mais relevantes na base de dados:')
mean_Xtfidf = X_tfidf.mean(axis=0)
sorted_meanXtfidf_indices = np.argsort(mean_Xtfidf)[::-1]
print(count_vect.get_feature_names_out()[sorted_meanXtfidf_indices[:3]])

Termos mais comuns nos 10 primeiros documentos:
['quadrinhos' 'histórias' 'ambiente']
['história' 'ensino' 'saberes']
['recursos' 'didáticos' 'aula']
['história' 'ensino' 'professores']
['história' 'disciplina' 'ensino']
['história' 'ensino' 'profissionais']
['história' 'cinema' 'realizadoras']
['aula' 'sala' 'biografias']
['tempo' 'histórico' 'professores']
['contadores' 'cearense' 'cariri']

Termos mais relevantes nos 10 primeiros documentos:
['quadrinhos' 'histórias' 'concepção']
['história' 'saberes' 'ensino']
['didáticos' 'recursos' 'aula']
['história' 'ensino' 'concepções']
['história' 'disciplina' 'ensino']
['história' 'profissionais' 'ensino']
['realizadoras' 'cinema' 'história']
['biografias' 'aula' 'sala']
['histórico' 'tempo' 'representação']
['contadores' 'cearense' 'cariri']

Termos mais relevantes na base de dados:
[['migrânea' 'estimulação' 'quantificação' 'forte' '1048692' 'estímulos'
  'universidade' 'atitudes' 'típica' 'mesma' 'motor' 'quociente'
  'negativa' 'nascida

## Criação de um transpose dataframe##

Todos os tokes viram linhas e os indices viram colunas. 

Agrupando Tema por média.

Renomeando as colunas.

In [33]:
def create_transpose_dataframe(X_tfidf):
  temp_df = pd.DataFrame(X_tfidf.toarray(), index=thesis['subject'], columns=count_vect.get_feature_names_out())
  temp_df = temp_df.groupby('subject').mean()
  temp_df = temp_df.stack().reset_index()
  temp_df = temp_df.rename(columns={0:'TFIDF', 'subject': 'Tema','level_1': 'Termo'})

  return temp_df

## Função que recebe um dataframe transposto, ordenação e retorna top 5 tokens

In [34]:
def get_most_relevant_terms(tfidf_df):
  top_terms = tfidf_df.sort_values(by=['Tema','TFIDF'], ascending=[True,False]).groupby(['Tema']).head(5)

  return top_terms

In [35]:
tfidf_df = create_transpose_dataframe(X_tfidf)
tfidf_df

,Tema,Termo,TFIDF
0,economia,1048689,0.000000
1,economia,1048692,0.000000
2,economia,1817,0.000000
3,economia,1990,0.009887
4,economia,1991,0.017070
...,...,...,...
4995,neuro+ciencia,índices,0.012059
4996,neuro+ciencia,órgão,0.023464
4997,neuro+ciencia,ótica,0.000000
4998,neuro+ciencia,última,0.003022


In [36]:
top_tfidf = get_most_relevant_terms(tfidf_df)
top_tfidf

,Tema,Termo,TFIDF
291,economia,economia,0.261067
877,economia,solidária,0.101092
304,economia,empresas,0.083653
701,economia,política,0.073341
755,economia,projeto,0.063250
1345,estatistica,estatística,0.194049
1914,estatistica,testes,0.106153
1297,estatistica,educação,0.083403
1314,estatistica,ensino,0.081566
1542,estatistica,matemática,0.073891


## Plot dos termos mais relevantes por tema (Altair)

In [37]:
import altair as alt
# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['TFIDF'] = top_tfidf_plusRand['TFIDF'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'Tema:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("TFIDF", order="descending")],
    groupby = ["Tema"],
).properties(
    title='5 tokens mais relevantes para cada tema'
)

# heatmap specification
heatmap = base.mark_rect().encode(
    #color = 'TFIDF:N',
    color = alt.Color(
            'TFIDF:Q',
            scale=alt.Scale(scheme='lightmulti')) 
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'Termo:N',
    color = alt.condition(alt.datum.TFIDF >= 0.2, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + text).properties(width =600, height=300).configure_title().configure_axis(
    labelFontSize = 12,
    titleFontSize = 14
)

alt.LayerChart(...)